<a href="https://colab.research.google.com/github/dcjshih/486-project/blob/master/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import os
os.chdir("/content/drive/MyDrive/Umich/IR Project")

import warnings
warnings.filterwarnings('ignore')

# Data Loading and Cleaning

In [6]:
neg_samples = []

In [46]:
with open("output_neg.txt", "r") as file:
    lines = file.readlines()
    for line in lines:
      all_words = line.split(",")
      neg_samples.append(all_words)

In [48]:
pos_samples = []

In [49]:
with open("output_pos.txt", "r") as file:
    lines = file.readlines()
    for line in lines:
      all_words = line.split(",")
      pos_samples.append(all_words)

In [23]:
import requests
from bs4 import BeautifulSoup

def get_website_text(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36'
    }
    try:
        response = requests.get(url, headers=headers, timeout=10)
        #response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        text = ' '.join([p.get_text() for p in soup.find_all('p')])
        return text
    except requests.exceptions.Timeout:
        print(f"Timeout occurred while fetching: {url}")
    except requests.exceptions.RequestException as e:
        print(f"Error fetching {url}: {e}")
    return ""

In [42]:
pos_samples[5][0].rsplit(":",1)[0]

'https://terrathread.com/collections/terra-thread-products'

In [32]:
pos_websitsText = dict()

for i in range(1, len(pos_samples)):
    website = pos_samples[i][0].rsplit(":",1)[0]
    pos_websitsText[website] = get_website_text(website)

In [40]:
pos_websitsText["https://us.boohoo.com/basic-rib-scoop-maxi-dress-/GZZ93943.html?color=165"]

"Hear about exclusive offers, get early access to collabs & quicker checkout   \n By providing your email address you are confirming that you are 16 or over and you are giving us consent to send you an email when the item becomes available. To find out more information about how to unsubscribe, please see our Privacy notice, Please do not subscribe if you are under 16. 60% cotton 35%polyester 5% elastane model wears US size 6 Our percentage off promotions, discounts, or sale markdowns are customarily based on our own opinion of the value of this product, which is not intended to reflect a former price at which this product has sold in the recent past. This amount represents our opinion of the full retail value of this product today based on our own assessment after considering a number of factors. That’s why before checking out, it’s important you acknowledge that you understand this. Cool with that? Great, happy shopping! Extended Christmas Returns - Any orders made between 11th Novem

In [43]:
neg_samples[5][0].rsplit(":",1)[0]

'https://terrathread.com/collections/terra-thread-products'

In [54]:
neg_websitsText = dict()

for i in range(1, len(neg_samples)):
    website = neg_samples[i][0].rsplit(":",1)[0]
    neg_websitsText[website] = get_website_text(website)

In [8]:
import json
filename = "pos.json"

with open(filename, 'w') as file:
    json.dump(pos_websitsText, file, indent=4)

filename = "neg.json"

with open(filename, 'w') as file:
    json.dump(neg_websitsText, file, indent=4)

In [12]:
import pandas as pd
df_pos = pd.read_json("pos.json")
df_pos.columns = ['url', 'text']
df_neg = pd.read_json("neg.json", orient='index').reset_index()
df_neg.columns = ['url', 'text']

In [59]:
df_pos["eco_friendly"] = "Yes"
df_neg["eco_friendly"] = "No"

In [78]:
for i in range(1, len(pos_samples)-1):
  if pos_samples[i][0].rsplit(":",1)[0] in df_pos["url"].values:
    df_pos.loc[df_pos["url"] == pos_samples[i][0].rsplit(":",1)[0], "text"] = df_pos.loc[df_pos["url"] == pos_samples[i][0].rsplit(":",1)[0], "text"] + " ".join(pos_samples[i+1][1:])

In [79]:
df_pos.head()

,url,text,eco_friendly
0,https://terrathread.com/,organic social impact ...,Yes
1,https://anchalproject.org/,...,Yes
2,https://us.boohoo.com/,"Hear about exclusive offers, get early access ...",Yes
3,https://terrathread.com/pages/terra-thread-shi...,...,Yes
4,https://terrathread.com/collections/terra-thre...,organic ...,Yes


In [80]:
for i in range(1, len(neg_samples)-1):
  if neg_samples[i][0].rsplit(":",1)[0] in df_neg["url"].values:
    df_neg.loc[df_neg["url"] == neg_samples[i][0].rsplit(":",1)[0], "text"] = df_neg.loc[df_neg["url"] == neg_samples[i][0].rsplit(":",1)[0], "text"] + " ".join(neg_samples[i+1][1:])

In [81]:
df_neg.head()

,url,text,eco_friendly
0,https://terrathread.com/,Country/region Country/region Featured in Kee...,No
1,https://anchalproject.org/,Set the table for a bright Easter brunch. Anch...,No
2,https://us.boohoo.com/,"Hear about exclusive offers, get early access ...",No
3,https://terrathread.com/pages/terra-thread-shi...,Country/region Country/region Our organic co...,No
4,https://terrathread.com/collections/terra-thre...,Country/region Country/region 21 products 21 p...,No


# Data Training - RUN FROM HERE

In [35]:
#all_df = pd.concat([df_pos, df_neg], ignore_index=True)
import pandas as pd
all_df = pd.read_csv("websites_data.csv")

In [21]:
#all_df.head()

In [22]:
unique_strings = []
with open("pos_keywords.txt", "r") as file:
    lines = file.readlines()
    for line in lines:
      unique_strings.append(line.strip())

In [23]:
eco_terms = {word: 2 for word in unique_strings}

In [24]:
all_df.head()

,url,text,eco_friendly
0,https://terrathread.com/,organic social impact ...,Yes
1,https://anchalproject.org/,...,Yes
2,https://us.boohoo.com/,"Hear about exclusive offers, get early access ...",Yes
3,https://terrathread.com/pages/terra-thread-shi...,...,Yes
4,https://terrathread.com/collections/terra-thre...,organic ...,Yes


In [25]:
website_embeddings = dict()
eco_status = dict()

In [26]:
all_df.head()

,url,text,eco_friendly
0,https://terrathread.com/,organic social impact ...,Yes
1,https://anchalproject.org/,...,Yes
2,https://us.boohoo.com/,"Hear about exclusive offers, get early access ...",Yes
3,https://terrathread.com/pages/terra-thread-shi...,...,Yes
4,https://terrathread.com/collections/terra-thre...,organic ...,Yes


In [28]:
import torch
from sentence_transformers import SentenceTransformer, util

# Load BERT model
model = SentenceTransformer('all-MiniLM-L6-v2')

def embed(text, is_eco_friendly):
      for i in unique_strings:
        if i in text:
          text += (" " + i) * int(eco_terms[i] * 2)

      # Convert to BERT embedding
      embedding = model.encode(text, convert_to_tensor=True)

      # Apply additional weight to eco-friendly websites
      if is_eco_friendly == "Yes":
          embedding *= 1.2

      return embedding

for _, row in all_df.iterrows():
    embedding = embed(row['text'], row['eco_friendly'])
    if embedding is not None:
        website_embeddings[row['url']] = embedding
        eco_status[row['url']] = row['eco_friendly']

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [31]:
def retrieve_websites(query, top_n=3):
    query_embedding = model.encode(query, convert_to_tensor=True)

    scores = {}
    for url, embedding in website_embeddings.items():
        similarity = util.pytorch_cos_sim(query_embedding, embedding).item()

        #Give eco-friendly websites a final priority boost
        if eco_status[url] == "Yes":
            similarity *= 1.2

        scores[url] = similarity

    # Sort by highest similarity
    top_websites = sorted(scores, key=scores.get, reverse=True)[:top_n]

    return top_websites

Top preferred websites: ['https://us.boohoo.com/crochet-maxi-cardigan/HZZ20847.html?color=123', 'https://anchalproject.org/collections/one-of-a-kind-quilted-jackets', 'https://us.boohoo.com/maxi-belted-trench-coat/HZZ25387.html', 'https://us.boohoo.com/super-soft-racer-neck-maxi-dress/GZZ98585.html?color=105', 'https://us.boohoo.com/basic-woven-relaxed-fit-crop-blazer/HZZ25532.html?color=186']


In [32]:
query = "sweater"
top_results = retrieve_websites(query, 5)

print("Top preferred websites:", top_results)

Top preferred websites: ['https://us.boohoo.com/crochet-maxi-cardigan/HZZ20847.html?color=123', 'https://anchalproject.org/collections/one-of-a-kind-quilted-jackets', 'https://us.boohoo.com/maxi-belted-trench-coat/HZZ25387.html', 'https://us.boohoo.com/super-soft-racer-neck-maxi-dress/GZZ98585.html?color=105', 'https://us.boohoo.com/basic-woven-relaxed-fit-crop-blazer/HZZ25532.html?color=186']


In [33]:
query = "jeans"
top_results = retrieve_websites(query, 5)

print("Top preferred websites:", top_results)

Top preferred websites: ['https://us.boohoo.com/basics-boyfriend-jeans/HZZ09457.html', 'https://us.boohoo.com/tailored-barrel-leg-trouser/HZZ25524.html?color=165', 'https://us.boohoo.com/fit-flare-tailored-trousers/HZZ25540.html?color=186', 'https://us.boohoo.com/fit-and-flare-tailored-trousers/HZZ25535.html?color=123', 'https://us.boohoo.com/suede-look-mini-skirt/HZZ18908.html']


In [34]:
query = "skirt"
top_results = retrieve_websites(query, 5)

print("Top preferred websites:", top_results)

Top preferred websites: ['https://us.boohoo.com/suede-look-mini-skirt/HZZ18908.html', 'https://us.boohoo.com/suede-look-mini-skirt/HZZ18908.html?color=186', 'https://us.boohoo.com/sand-denim-mini-skirt/HZZ03307.html', 'https://us.boohoo.com/woven-tailored-short/HZZ25528.html?color=440', 'https://us.boohoo.com/super-soft-racer-neck-maxi-dress/GZZ98585.html?color=105']
